# PyTorch Tutorial - Part 4
> "Part 4 of the PyTorch tutorial"

- toc: true
- branch: master
- badges: true
- comments: true
- author: Scott Wolf
- categories: [PyTorch, jupyter]

# Building a model!

Finally -- we can build the model that PyTorch promises. Sure, there have been a few extra steps relative to Keras/TF, but here is where PyTorch shines.

In [6]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Defining our model class
Here we just need to implement `__init__` to actually build the model and `__forward__` which actually implements the model.

In [10]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__() # Call nn.Module init!
    self.flatten = nn.Flatten() # Smash the picture to 1d!
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512), # Hidden! Input length is just 28x28 because that's the image size! For now, we'll just make the output size 512.
        nn.ReLU(),
        nn.Linear(512,512), # Hidden!
        nn.ReLU(),
        nn.Linear(512,10) # our 512 vec and put it into our output class size(10)!
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits


model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='cuda:0')


In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0255, -0.0169, -0.0031,  ...,  0.0331, -0.0003,  0.0136],
        [ 0.0226, -0.0092, -0.0174,  ..., -0.0345,  0.0264, -0.0332]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0015, -0.0132], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0185,  0.0232,  0.0248,  ..., -0.0187, -0.0252, -0.0192],
        [ 0.0440,  0.0210, -0.0330,  ..., -0.0280, -0.0106, -0.0195]],
       device='cuda:0', grad_fn=<Sl